<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3

### Iowa Liquor Sales 

---

Project 3, unlike project 2 which combined skills from weeks 1 and 2, will now be review of week 2. Projects going forward will fit this format: you learn material one week and review it through a project the following week.

That being said, this is as you might expect more challenging. You will be using the pandas data transformation functionality (melt, pivot, groupby, apply) that was not a factor in Project 2.

---

#### Context of the data

The state of Iowa provides many data sets on their website, including [this dataset](https://www.dropbox.com/sh/pf5n5sgfgiri3i8/AACkaMeL_i_WgZ00rpxOOcysa?dl=0) which contains transactions for all stores that have a class E liquor license. You can choose one of the following two scenarios.

The data can also be found [directly on their website](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy), which allows you to explore it graphically and download it (though it doesn't work very well).

NOTE: Some of you may have computer issues with the full dataset. In this case, feel free to use [this 10% dataset version of Iowa liquor sales](https://drive.google.com/file/d/0Bx2SHQGVqWaseDB4QU9ZSVFDY2M/view?usp=sharing). You may want to use it anyway to test and prototype your code since it will be faster, before running it on the full dataset.

---

#### Package imports

In [3]:
import numpy as np
import scipy.stats as stats
import pandas as pd

# this line tells jupyter notebook to put the plots in the notebook rather than saving them to file.
%matplotlib inline

# this line makes plots prettier on mac retina screens. If you don't have one it shouldn't do anything.
%config InlineBackend.figure_format = 'retina'

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Load the data from file

---

In [4]:
iowa_file = '~/Desktop/DSI/DSI-SF-2/datasets/iowa_liquor/Iowa_Liquor_sales_sample_10pct.csv'

iowa = pd.read_csv(iowa_file)

In [129]:
iowa.head(3)

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),broad_type
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,4.50,6.75,12,81.00,9.0,2.38,BRANDY
1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,13.75,20.63,2,41.26,1.5,0.40,WHISKEY
2,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,12.59,18.89,24,453.36,24.0,6.34,WHISKEY


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Do an initial overview of the data

---

At the very least describe the columns/variables and the datatypes. 

In [6]:
iowa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270955 entries, 0 to 270954
Data columns (total 18 columns):
Date                     270955 non-null object
Store Number             270955 non-null int64
City                     270955 non-null object
Zip Code                 270955 non-null object
County Number            269878 non-null float64
County                   269878 non-null object
Category                 270887 non-null float64
Category Name            270323 non-null object
Vendor Number            270955 non-null int64
Item Number              270955 non-null int64
Item Description         270955 non-null object
Bottle Volume (ml)       270955 non-null int64
State Bottle Cost        270955 non-null object
State Bottle Retail      270955 non-null object
Bottles Sold             270955 non-null int64
Sale (Dollars)           270955 non-null object
Volume Sold (Liters)     270955 non-null float64
Volume Sold (Gallons)    270955 non-null float64
dtypes: float64(4), int64(

In [7]:
iowa.shape

(270955, 18)

In [8]:
iowa.columns

Index([u'Date', u'Store Number', u'City', u'Zip Code', u'County Number',
       u'County', u'Category', u'Category Name', u'Vendor Number',
       u'Item Number', u'Item Description', u'Bottle Volume (ml)',
       u'State Bottle Cost', u'State Bottle Retail', u'Bottles Sold',
       u'Sale (Dollars)', u'Volume Sold (Liters)', u'Volume Sold (Gallons)'],
      dtype='object')

In [9]:
iowa.dtypes

Date                      object
Store Number               int64
City                      object
Zip Code                  object
County Number            float64
County                    object
Category                 float64
Category Name             object
Vendor Number              int64
Item Number                int64
Item Description          object
Bottle Volume (ml)         int64
State Bottle Cost         object
State Bottle Retail       object
Bottles Sold               int64
Sale (Dollars)            object
Volume Sold (Liters)     float64
Volume Sold (Gallons)    float64
dtype: object

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Clean the dataset

---

### 3.1 Identify columns that you will need to convert and clean. Where and how is the data corrupted?

Don't worry about converting the date column to a pandas/numpy "datetime" datatype, unless you want to (not required for these problems and is a challenging thing to work with in its own right.)

In [10]:
iowa.isnull().sum()

Date                        0
Store Number                0
City                        0
Zip Code                    0
County Number            1077
County                   1077
Category                   68
Category Name             632
Vendor Number               0
Item Number                 0
Item Description            0
Bottle Volume (ml)          0
State Bottle Cost           0
State Bottle Retail         0
Bottles Sold                0
Sale (Dollars)              0
Volume Sold (Liters)        0
Volume Sold (Gallons)       0
dtype: int64

- City has uppercase and lowercase same cities
- Category Name needs a broader name
- Remove the $ sign in Sales (Dollars)
- Change date column to 'datetime' column

### 3.2 Clean the columns

In [52]:
# Changed all city names from lowercase to uppercase
def lower_to_upper(input):
    return input.upper()

iowa['City'] = iowa['City'].apply(lower_to_upper)

In [68]:
# Category Name fill nans with 'UNLISTED'
iowa['Category Name'] = iowa['Category Name'].fillna('UNLISTED',axis=0)
iowa['Category Name'].unique()

array(['APRICOT BRANDIES', 'BLENDED WHISKIES', 'STRAIGHT BOURBON WHISKIES',
       'AMERICAN COCKTAILS', 'VODKA 80 PROOF', 'AMERICAN DRY GINS',
       'AMERICAN GRAPE BRANDIES', 'CANADIAN WHISKIES', 'IMPORTED VODKA',
       'CINNAMON SCHNAPPS', 'SINGLE BARREL BOURBON WHISKIES',
       'PEPPERMINT SCHNAPPS', 'VODKA FLAVORED', '100 PROOF VODKA',
       'DISTILLED SPIRITS SPECIALTY', 'TEQUILA', 'TENNESSEE WHISKIES',
       'WHISKEY LIQUEUR', 'MISC. AMERICAN CORDIALS & LIQUEURS',
       'APPLE SCHNAPPS', 'CHERRY BRANDIES',
       'MISC. IMPORTED CORDIALS & LIQUEURS', 'IMPORTED DRY GINS',
       'WATERMELON SCHNAPPS', 'BLACKBERRY BRANDIES', 'SINGLE MALT SCOTCH',
       'PUERTO RICO & VIRGIN ISLANDS RUM', 'SPICED RUM', 'CREAM LIQUEURS',
       'SCOTCH WHISKIES', 'IMPORTED VODKA - MISC', 'IRISH WHISKIES',
       'FLAVORED RUM', 'IMPORTED GRAPE BRANDIES', 'SPEARMINT SCHNAPPS',
       'RASPBERRY SCHNAPPS', 'TRIPLE SEC', 'AMERICAN AMARETTO',
       'IMPORTED SCHNAPPS', 'STRAIGHT RYE WHISKIES', '

In [366]:
def alcohol_cat(alcohols):
    if 'SCHNAPPS' in alcohols:
        return 'SCHNAPP'
    elif ('WHISKIES' in alcohols) or ('WHISKY' in alcohols) or ('WHISKEY' in alcohols):
        return 'WHISKEY'
    elif ('GINS' in alcohols) or ('GIN' in alcohols):
        return 'GIN'
    elif 'RUM' in alcohols:
        return 'RUM'
    elif ('BRANDIES' in alcohols) or ('BRANDY' in alcohols):
        return 'BRANDY'
    elif 'VODKA' in alcohols:
        return 'VODKA'
    elif ('BEERS' in alcohols) or ('BEER' in alcohols):
        return 'BEER'
    elif ('TEQUILA' in alcohols):
        return 'TEQUILA'
    else:
        return 'OTHER'

iowa['broad_type'] = iowa['Category Name'].map(alcohol_cat)
print iowa.broad_type.value_counts()

WHISKEY    80561
VODKA      68868
OTHER      34281
RUM        22362
GIN        19691
BRANDY     14726
SCHNAPP    14354
TEQUILA    11936
BEER           2
Name: broad_type, dtype: int64


In [74]:
# Remove $ sign from Sale (Dollars) and change to float
def remove_dollar(input):
    return float(input.replace('$',''))

iowa['Sale (Dollars)'] = iowa['Sale (Dollars)'].apply(remove_dollar)
iowa['Sale (Dollars)'].mean()

128.90237474857972

In [89]:
# Remove State Bottle Cost/Retail $ sign
iowa['State Bottle Cost'] = iowa['State Bottle Cost'].apply(remove_dollar)
iowa['State Bottle Retail'] = iowa['State Bottle Retail'].apply(remove_dollar)

In [91]:
print iowa['State Bottle Cost'].mean()
print iowa['State Bottle Retail'].mean()

9.77328091381
14.6757599601


In [80]:
# Convert Date object to Datetime
iowa['Date'] = pd.to_datetime(iowa['Date'], infer_datetime_format=True)

0   2015-11-04
1   2016-03-02
2   2016-02-11
3   2016-02-03
4   2015-08-18
Name: Date, dtype: datetime64[ns]

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 3.3 Perform more extensive cleaning of the dataset

Cleaning of data can mean a lot more than just fixing strings and numbers in columns. There are often logical errors with data, useless or nonsensical categories, redundancy of information, outliers, and many more problems.

This dataset has problems beyond just fixing the types of columns. Though resolving them may not be required for EDA and analysis, if you want experience with "deeper" cleaning of data this is a great dataset to start practicing with.

Keep in mind that some types of "data cleaning" is subjective: it's not always a cut-and-dry conversion of type or removal of null values. Subjectivity when dealing with data is just a fact of life for a data scientist. This isn't a kind of programming where things are just right or wrong.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Filter/adjust the store data

---

Some stores may have opened or closed in 2015. These stores will have incorrect yearly summary statistics since they were not open the full year. We need to filter them out or find another way to deal with the inconsistent numbers of months across stores.

It is up to you how you want to deal with this problem.

1. Investigate problematic stores.
2. Decide on an approach to deal with stores not open for the full 2015 year.
3. Implement your plan.
4. Briefly report on what you did and why.


In [221]:
iowa_min = iowa.groupby(['Store Number']).min().sort_values('Date').rename(columns={'Date':'date_min'}).reset_index()

In [227]:
iowa_min = iowa_min[['Store Number', 'date_min']]
iowa_min.head(3)

,Store Number,date_min
0,4540,2015-01-05
1,4000,2015-01-05
2,2962,2015-01-05


In [228]:
iowa_max = iowa.groupby(['Store Number']).max().sort_values('Date').rename(columns={'Date':'date_max'}).reset_index()

In [241]:
iowa_max = iowa_max[['Store Number', 'date_max']]
iowa_max.tail(3)

,Store Number,date_max
1397,4969,2016-03-31
1398,4902,2016-03-31
1399,2106,2016-03-31


In [234]:
iowa_minmax = iowa_min.merge(iowa_max, how='outer', on='Store Number')

In [325]:
badstore_mask = (iowa_minmax['date_min'] < '2015-02-01') & (iowa_minmax['date_max'] < '2015-12-01')
iowa_badstores = iowa_minmax[badstore_mask]
iowa_badstores.head()

,Store Number,date_min,date_max
2,2962,2015-01-05,2015-03-26
3,2963,2015-01-05,2015-03-19
14,2959,2015-01-05,2015-03-25
30,2958,2015-01-05,2015-02-05
45,4870,2015-01-05,2015-02-23


In [326]:
badstores = []
for x in iowa_badstores['Store Number']:
    badstores.append(x)

In [327]:
for bad in badstores:
    iowa = iowa[iowa['Store Number'] != bad]
iowa.head(3)

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,...,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),broad_type,profit_dollars,profit_per_gallon
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,...,750,4.50,6.75,12,81.00,9.0,2.38,BRANDY,27.00,22.689076
1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,...,750,13.75,20.63,2,41.26,1.5,0.40,WHISKEY,13.76,68.750000
2,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,...,1000,12.59,18.89,24,453.36,24.0,6.34,WHISKEY,151.20,47.659306


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Proposals for new tax rates (Scenario 1)

---

You are a data scientist in residence at the Iowa State tax board. The Iowa State legislature is considering changes in the liquor tax rates and has assigned you to the project.

### 5.1 Calculate yearly liquor sales for each store in 2015.

In [525]:
date_mask = (iowa['Date'] > '2015-01-01') & (iowa['Date'] < '2016-01-01')
iowa_annual_sales = iowa[date_mask].groupby(['Store Number'])[['Sale (Dollars)']].sum().reset_index()
iowa_annual_sales.sort_values('Sale (Dollars)', ascending=False).head()

,Store Number,Sale (Dollars)
121,2633,1000783.30
971,4829,878812.70
180,3385,441983.55
33,2512,419746.83
186,3420,352020.66


### 5.2 Calculate the profit each store is making in 2015.


In [331]:
iowa['profit_dollars'] = iowa['Sale (Dollars)'] - (iowa['State Bottle Cost'] * iowa['Bottles Sold'])

In [480]:
iowa_annual_profit = iowa[date_mask].groupby(['Store Number'])[['profit_dollars', 'Sale (Dollars)']].sum().reset_index()
iowa_annual_profit.sort_values('profit_dollars', ascending=False).head()

,Store Number,profit_dollars,Sale (Dollars)
121,2633,333816.91,1000783.30
971,4829,293196.99,878812.70
180,3385,147884.62,441983.55
33,2512,140143.46,419746.83
186,3420,117593.34,352020.66


### 5.3 Investigate which Iowa counties are making the most profit on liquor per gallon in 2015.

In [333]:
iowa['profit_per_gallon'] = iowa['profit_dollars'] / iowa['Volume Sold (Gallons)']

In [334]:
iowa_per_gallon = iowa[date_mask].groupby(['County'])[['profit_per_gallon']].sum().reset_index()
iowa_per_gallon.sort_values('profit_per_gallon', ascending=False).head()

,County,profit_per_gallon
76,Polk,961990.228053
56,Linn,435983.482090
81,Scott,317522.447266
6,Black Hawk,280972.997051
51,Johnson,257205.453714


<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.4 Plot a color map of average profit per county in 2015.

[Check out this site](http://flowingdata.com/2009/11/12/how-to-make-a-us-county-thematic-map-using-free-tools/) for  information on how to create map visualizations in python. There are other resources online for map plotting in python as well.

Another interesting map plot might show the difference in profit between time periods. Change in profit from November to December, for example.

In [335]:
iowa_avg_profit = iowa[date_mask].groupby(['County'])[['profit_dollars']].mean().reset_index()

In [522]:
import csv
from bs4 import BeautifulSoup

counties_path = '/Users/edwardlee/Desktop/DSI-SF-2-azedlee/counties.svg'

svg = open(counties_path, 'r').read()

# Load into Beautiful Soup
soup = BeautifulSoup(svg, selfClosingTags=['defs','sodipodi:namedview'])

# Find counties
paths = soup.findAll('path')

# Map colors
colors = ["#F1EEF6", "#D4B9DA", "#C994C7", "#DF65B0", "#DD1C77", "#980043"]

# County style
'font-size:12px;fill-rule:nonzero;stroke:#FFFFFF;stroke-opacity:1;
stroke-width:0.1;stroke-miterlimit:4;stroke-dasharray:none;stroke-linecap:butt;
marker-start:none;stroke-linejoin:bevel;fill:'

            

SyntaxError: EOL while scanning string literal (<ipython-input-522-ca5a0a8b6a11>, line 18)

### 5.5  Does profit scale proportionally to store size?

Size of a store can be defined as the total gallons sold in 2015. Is the profit to store size ratio increasing as store size increases? Decreasing? Consistent?

In [336]:
iowa_tot_gallon = iowa[date_mask].groupby(['Store Number'])[['profit_dollars','Volume Sold (Gallons)']].sum().reset_index()
iowa_tot_gallon.sort_values('profit_dollars', ascending=False)
iowa_tot_gallon['profit_gallon_ratio'] = iowa_tot_gallon['profit_dollars'] / iowa_tot_gallon['Volume Sold (Gallons)']
iowa_tot_gallon = iowa_tot_gallon.sort_values('profit_gallon_ratio', ascending=False)
iowa_tot_gallon

,Store Number,profit_dollars,Volume Sold (Gallons),profit_gallon_ratio
834,4669,3143.12,36.39,86.373179
1277,9001,9403.68,168.83,55.699106
985,4849,3332.80,65.81,50.642759
1276,5215,18.68,0.48,38.916667
1278,9002,12574.08,328.17,38.315751
1249,5175,74.99,2.05,36.580488
1161,5074,1038.87,28.43,36.541330
920,4766,480.95,13.48,35.678783
1279,9010,1920.12,56.17,34.184084
1112,5014,1790.36,52.70,33.972676


In [337]:
iowa_tot_gallon.describe()

,Store Number,profit_dollars,Volume Sold (Gallons),profit_gallon_ratio
count,1281.000000,1281.000000,1281.000000,1281.000000
mean,4235.722092,7348.399321,404.125246,17.874384
std,831.933091,17530.743437,877.097361,4.346021
min,2106.000000,13.340000,0.480000,10.025417
25%,3855.000000,1165.170000,68.140000,15.331769
50%,4436.000000,2585.310000,151.950000,17.302230
75%,4813.000000,6784.460000,393.860000,19.390057
max,9013.000000,333816.910000,16356.020000,86.373179


# RESULT
From the results, it seems that the smaller stores have a higher profit to store size ratio. Although smaller stores do seem to have a better profit to gallon ratio, the mean and median are rather close, which is pretty normally distributed. The max ratio is more than 3 standard deviations away (also happens to be the ranked first), however, below the 25th percentile in volume sold and above the median for profit dollars. It may be interesting to see whether this data point is correct or how their sales model works.

### 5.6 Create a broader category for liquor type.

Liquor types are pretty granular in this dataset. Create a column that categorizes these types into a smaller amount of categories. The categories you decide on are up to you.

In [328]:
### Refer to 3.2

### 5.7 Do relationships exist between the broader liquor type and profit?

In [329]:
iowa.head(1)

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,...,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),broad_type,profit_dollars,profit_per_gallon
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,...,750,4.5,6.75,12,81.0,9.0,2.38,BRANDY,27.0,22.689076


In [367]:
iowa_alc_type = iowa.groupby(['broad_type'])[['State Bottle Cost','State Bottle Retail','Bottles Sold','profit_dollars']].mean().reset_index()
iowa_alc_type['cost_retail_ratio'] = iowa_alc_type['State Bottle Retail'] / iowa_alc_type['State Bottle Cost']
iowa_alc_type.sort_values('cost_retail_ratio', ascending=False)

,broad_type,State Bottle Cost,State Bottle Retail,Bottles Sold,profit_dollars,cost_retail_ratio
2,GIN,8.211458,12.332931,8.891880,33.013382,1.501917
7,VODKA,8.028544,12.057905,11.460475,40.128317,1.501880
3,OTHER,11.458206,17.208762,7.754383,39.656070,1.501872
8,WHISKEY,11.196157,16.814734,10.026167,50.839910,1.501831
1,BRANDY,8.247129,12.379368,9.888768,32.250912,1.501052
4,RUM,9.300986,13.960180,10.515562,51.987539,1.500936
6,TEQUILA,13.720637,20.592052,9.408847,52.302766,1.500809
5,SCHNAPP,7.466947,11.202626,7.578863,29.257731,1.500295
0,BEER,95.170000,142.760000,1.000000,47.590000,1.500053


In [368]:
iowa_alc_type = iowa.groupby(['broad_type'])[['State Bottle Cost','State Bottle Retail','Bottles Sold','profit_dollars']].sum().reset_index()
iowa_alc_type.sort_values('profit_dollars', ascending=False)

,broad_type,State Bottle Cost,State Bottle Retail,Bottles Sold,profit_dollars
8,WHISKEY,901973.61,1354611.80,807718,4.095714e+06
7,VODKA,552909.74,830403.81,789260,2.763557e+06
3,OTHER,392798.75,589933.56,265828,1.359450e+06
4,RUM,207988.65,312177.55,235149,1.162545e+06
2,GIN,161691.82,242847.74,175090,6.500665e+05
6,TEQUILA,163769.52,245786.73,112304,6.242858e+05
1,BRANDY,121447.22,182298.57,145622,4.749269e+05
5,SCHNAPP,107180.56,160802.50,108787,4.199655e+05
0,BEER,190.34,285.52,2,9.518000e+01


# RESULT
Although Whiskey is ranked first in profit dollars and bottles sold, Tequila actually edges Whiskey out in average profit dollars at the same average cost per retail ratio. However, in general, Whiskey is still the best selling alcohol type and Tequila has the best best average profit.


<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.8 Are there any noticeable relationships between liquor brand and sales? Profit?

In [363]:
iowa_alc_type = iowa.groupby(['broad_type'])[['State Bottle Cost','State Bottle Retail','Sale (Dollars)','profit_dollars']].sum().reset_index()
iowa_alc_type['sales_profit_ratio'] = iowa_alc_type['Sale (Dollars)'] / iowa_alc_type['profit_dollars']
iowa_alc_type.sort_values('profit_dollars', ascending=False)

,broad_type,State Bottle Cost,State Bottle Retail,Sale (Dollars),profit_dollars,sales_profit_ratio
7,WHISKEY,901973.61,1354611.80,1.222638e+07,4.095714e+06,2.985164
6,VODKA,552909.74,830403.81,8.269190e+06,2.763557e+06,2.992227
3,OTHER,556568.27,835720.29,5.939102e+06,1.983736e+06,2.993898
4,RUM,207988.65,312177.55,3.484830e+06,1.162545e+06,2.997586
2,GIN,161691.82,242847.74,1.945553e+06,6.500665e+05,2.992852
1,BRANDY,121447.22,182298.57,1.423093e+06,4.749269e+05,2.996445
5,SCHNAPP,107180.56,160802.50,1.259460e+06,4.199655e+05,2.998962
0,BEER,190.34,285.52,2.855200e+02,9.518000e+01,2.999790


# RESULT
Surprising, the sales profit ratio is quite even for all types. However, it is clear that Iowa loves their Whiskey and is by far the best seller in sales and profit. I would love to get more information on Beer, which may be in the full dataset, but I assume that Beer would rank in the top 3 with more data points.

### 5.9 The tax board wants to design a tax or taxes that affect larger stores more than smaller "mom and pop" stores.

Based on your investigations into the data, come up with a way you could design a tax that achieves this goal **without explicitly taxing stores based on size or county critera.** The liqour board does not want to obviously punish larger stores or speific counties for fear of backlash, but is willing to tax hard alcohol more than beer, for example.

Feel free to do more EDA if it helps.

Your report should describe whether such a tax is possible or not, and the specifics of what the tax will target/do.

In [407]:
smaller_mask = iowa_tot_gallon['Volume Sold (Gallons)'] < 151.95
larger_mask = iowa_tot_gallon['Volume Sold (Gallons)'] > 151.95
iowa_tot_gallon[smaller_mask].sort_values('profit_gallon_ratio', ascending=False).head(10)

,Store Number,profit_dollars,Volume Sold (Gallons),profit_gallon_ratio
834,4669,3143.12,36.39,86.373179
985,4849,3332.80,65.81,50.642759
1276,5215,18.68,0.48,38.916667
1249,5175,74.99,2.05,36.580488
1161,5074,1038.87,28.43,36.541330
920,4766,480.95,13.48,35.678783
1279,9010,1920.12,56.17,34.184084
1112,5014,1790.36,52.70,33.972676
1202,5118,204.94,6.35,32.274016
1107,5009,1352.29,42.01,32.189717


In [408]:
iowa_tot_gallon[larger_mask].sort_values('profit_gallon_ratio', ascending=False).head(10)

,Store Number,profit_dollars,Volume Sold (Gallons),profit_gallon_ratio
1277,9001,9403.68,168.83,55.699106
1278,9002,12574.08,328.17,38.315751
1081,4979,10631.11,333.73,31.855422
815,4648,16140.07,524.31,30.783449
11,2248,22584.51,743.70,30.367769
198,3521,13636.49,453.87,30.044925
253,3698,4520.79,152.23,29.697103
1212,5131,5524.18,189.57,29.140581
297,3805,12795.59,445.74,28.706398
807,4638,11915.13,417.02,28.572083


In [378]:
iowa_tot_gallon.describe()

,Store Number,profit_dollars,Volume Sold (Gallons),profit_gallon_ratio
count,1281.000000,1281.000000,1281.000000,1281.000000
mean,4235.722092,7348.399321,404.125246,17.874384
std,831.933091,17530.743437,877.097361,4.346021
min,2106.000000,13.340000,0.480000,10.025417
25%,3855.000000,1165.170000,68.140000,15.331769
50%,4436.000000,2585.310000,151.950000,17.302230
75%,4813.000000,6784.460000,393.860000,19.390057
max,9013.000000,333816.910000,16356.020000,86.373179


In [396]:
iowa_tax = iowa.groupby(['broad_type'])[['Volume Sold (Gallons)','State Bottle Cost','State Bottle Retail','Sale (Dollars)','profit_dollars']].mean().reset_index()
iowa_tax.sort_values('profit_dollars', ascending=False)

,broad_type,Volume Sold (Gallons),State Bottle Cost,State Bottle Retail,Sale (Dollars),profit_dollars
6,TEQUILA,2.041556,13.720637,20.592052,156.745718,52.302766
4,RUM,2.675042,9.300986,13.960180,155.837113,51.987539
8,WHISKEY,2.502102,11.196157,16.814734,151.765447,50.839910
0,BEER,0.200000,95.170000,142.760000,142.760000,47.590000
7,VODKA,2.793340,8.028544,12.057905,120.073033,40.128317
3,OTHER,1.997793,11.458206,17.208762,118.671727,39.656070
2,GIN,2.242364,8.211458,12.332931,98.804174,33.013382
1,BRANDY,1.322176,8.247129,12.379368,96.638099,32.250912
5,SCHNAPP,1.781508,7.466947,11.202626,87.742822,29.257731


In [420]:
iowa_tax = iowa[date_mask].groupby(['Store Number'])[['profit_dollars','Volume Sold (Gallons)', 'Bottles Sold']].mean().reset_index()
iowa_tax.sort_values('Volume Sold (Gallons)', ascending=False).head(10)
# iowa_tax['profit_gallon_ratio'] = iowa_tax['profit_dollars'] / iowa_tax['Volume Sold (Gallons)']
# iowa_tax = iowa_tot_gallon.sort_values('profit_gallon_ratio', ascending=False)
# iowa_tax
iowa_tax.describe()

,Store Number,profit_dollars,Volume Sold (Gallons),Bottles Sold
count,1281.000000,1281.000000,1281.000000,1281.000000
mean,4235.722092,41.116364,2.338648,10.570780
std,831.933091,38.868292,1.872232,6.985107
min,2106.000000,6.670000,0.240000,1.500000
25%,3855.000000,25.445515,1.453882,6.703704
50%,4436.000000,35.137522,2.031429,9.488844
75%,4813.000000,48.090000,2.760000,12.375000
max,9013.000000,691.704000,28.583917,87.266667


# Report

A possible tax would be the amount of bottles bought by the stores. Larger stores purchase more bottles monthly compared to smaller stores and have an average higher profit_dollars and average higher bottles sold. Depending on how many bottles bought and what type of alcohol, the tax can be sorted by how many standard deviations away from the bottles bought to sold.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. Market research for new store locations (Scenario 2)

A liquor store owner in Iowa is looking to open a store in a new location and has hired you to investigate the market data for potential new locations. The business owner is interested in what you can discover from the data that will help him evaluate a good location for a new storefront.

---

### 6.1 Create columns for Q1 2015 sales and Q1 2016 sales.

The owner is not just interested in what locations are currently selling well, but also which areas are showing the most promising sales growth from 2015 to 2016. 

For yearly changes we will focus on the changes from Q1 2015 to Q1 2016 specifically (avoiding effects of season changes).

Note: Q1 is months: January, February, and March.


In [471]:
q1_2015 = (iowa['Date'] >= '01-01-2015') & (iowa['Date'] < '04-01-2015')
q1_2016 = (iowa['Date'] >= '01-01-2016') & (iowa['Date'] < '04-01-2016')
iowa_q1_2015_tot = iowa[q1_2015].groupby(['County'])[['Sale (Dollars)']].sum().reset_index().rename(columns={'Sale (Dollars)':'2015 Q1 Total'})
iowa_q1_2015_avg = iowa[q1_2015].groupby(['County'])[['Sale (Dollars)']].mean().reset_index().rename(columns={'Sale (Dollars)':'2015 Q1 Avg'})

In [472]:
iowa_q1_2016_tot = iowa[q1_2016].groupby(['County'])[['Sale (Dollars)']].sum().reset_index().rename(columns={'Sale (Dollars)':'2016 Q1 Total'})
iowa_q1_2016_avg = iowa[q1_2016].groupby(['County'])[['Sale (Dollars)']].mean().reset_index().rename(columns={'Sale (Dollars)':'2016 Q1 Avg'})

In [473]:
iowa_2015_2016 = iowa_q1_2015_tot.merge(iowa_q1_2016_tot,on='County').merge(iowa_q1_2015_avg,on='County').merge(iowa_q1_2016_avg,on='County')

In [474]:
iowa_2015_2016['2015_2016_growth_%'] = ((iowa_2015_2016['2016 Q1 Total'] - iowa_2015_2016['2015 Q1 Total']) / iowa_2015_2016['2015 Q1 Total']) * 100
iowa_2015_2016.sort_values('2015_2016_growth_%', ascending=False).head(10)

,County,2015 Q1 Total,2016 Q1 Total,2015 Q1 Avg,2016 Q1 Avg,2015_2016_growth_%
4,Audubon,768.96,3226.28,96.120000,89.618889,319.564087
86,Taylor,1459.28,3215.20,76.804211,46.597101,120.327833
32,Fayette,18185.53,37743.63,116.573910,131.054271,107.547594
23,Crawford,18823.90,37830.13,93.651244,200.159418,100.968609
15,Cedar,9921.67,19886.09,55.428324,101.459643,100.430875
97,Worth,6046.25,11807.91,90.242537,135.723103,95.293116
12,Calhoun,3971.11,7415.91,65.100164,85.240345,86.746527
11,Butler,4694.47,8583.26,66.119296,95.369556,82.837679
46,Ida,8080.16,13303.26,78.448155,113.703077,64.641047
17,Cherokee,10347.10,15362.56,98.543810,97.231392,48.472132


### 6.2 Decide on a metric for performance that combines Q1 2016 sales as well as growth in sales from Q1 2015.

Again, the owner wants to know that his store will be in a good position to sell a large amount of liquor now, but also have his store be more likely to grow and sell even more going forward. 

In particular, he is worried about opening a store in an area that is showing a decline in liquor sales between Q1 2015 and Q1 2016.

In [475]:
iowa_2015_2016.sort_values('2015_2016_growth_%', ascending=False).tail(10)

,County,2015 Q1 Total,2016 Q1 Total,2015 Q1 Avg,2016 Q1 Avg,2015_2016_growth_%
68,Montgomery,16588.99,12560.33,112.850272,93.733806,-24.285143
13,Carroll,55336.66,38227.43,175.116013,120.591262,-30.918436
67,Monroe,6957.00,4508.16,96.625000,81.966545,-35.199655
42,Harrison,14291.40,9217.60,57.626613,51.784270,-35.502470
75,Pocahontas,13525.28,8622.08,127.596981,82.115048,-36.252115
82,Shelby,18156.85,9261.05,126.089236,82.687946,-48.994181
18,Chickasaw,11081.11,5590.89,116.643263,74.545200,-49.545759
60,Madison,35020.89,15029.29,168.369663,91.642012,-57.084786
34,Franklin,12709.25,4980.36,78.939441,75.460000,-60.813109
44,Howard,30923.00,8235.96,211.801370,105.589231,-73.366232


### 6.3 Recommend a location based on your metric.

The owner asks you to make a recommendation for a new store location. Explain why you recommended this area.

# RECOMMENDATION
I would recommend Fayette or Crawford as a new store location. Both locations have over a 100% growth rate since 2015 Q1 and have a large quantity of a significant sales total at 30k in 3 months, which is the largest in the top 10 highest growth rate. Audobon and Taylor do have higher growth rates, but their total sales of 3k in 3 months concerns the buy rate. Fayette or Crawford also are growing counties, which is reasonable to open a new store location.

### 6.4 Recommend a location based also on a restricted inventory type.

Your employer has now decided he wants his store to focus on one type of liquor only. He askes you:

1. What would be the best type of liquor to sell to maximize profit?
2. Should the recommended location change if he is only selling one type of liquor?

In [500]:
iowa_q1_2015_tot_type = iowa[q1_2015].groupby(['broad_type','County'])[['Sale (Dollars)']].sum().reset_index().rename(columns={'Sale (Dollars)':'2015 Q1 Total'})
iowa_q1_2016_tot_type = iowa[q1_2016].groupby(['broad_type','County'])[['Sale (Dollars)']].sum().reset_index().rename(columns={'Sale (Dollars)':'2016 Q1 Total'})

In [508]:
result = pd.merge(iowa_q1_2015_tot_type, iowa_q1_2016_tot_type, on=['broad_type','County'])

In [512]:
result['growth_%'] = ((result['2016 Q1 Total'] - result['2015 Q1 Total'])/result['2015 Q1 Total'])*100
result.sort_values('growth_%', ascending=False).head(10)

,broad_type,County,2015 Q1 Total,2016 Q1 Total,growth_%
666,WHISKEY,Audubon,49.62,789.66,1491.414752
328,RUM,Ida,181.53,2694.74,1384.459869
559,TEQUILA,Winnebago,31.48,456.54,1350.254130
517,TEQUILA,Ida,41.76,535.88,1183.237548
424,SCHNAPP,Ida,51.00,582.91,1042.960784
25,BRANDY,Decatur,19.32,202.92,950.310559
263,OTHER,Ringgold,37.77,374.74,892.163092
499,TEQUILA,Decatur,33.00,273.63,729.181818
295,RUM,Calhoun,158.49,1255.44,692.125686
475,TEQUILA,Adair,127.56,998.86,683.051113


In [514]:
# fayette mask
fayette_mask = (result['County'] == 'Fayette')
result[fayette_mask].sort_values('growth_%', ascending=False)

,broad_type,County,2015 Q1 Total,2016 Q1 Total,growth_%
505,TEQUILA,Fayette,875.28,3822.65,336.734531
595,VODKA,Fayette,2220.68,7936.99,257.412594
30,BRANDY,Fayette,388.58,1067.37,174.684750
694,WHISKEY,Fayette,7881.55,15720.38,99.457975
315,RUM,Fayette,2389.17,3997.49,67.317102
217,OTHER,Fayette,1824.28,2497.26,36.890170
120,GIN,Fayette,797.44,966.27,21.171499
412,SCHNAPP,Fayette,1808.55,1735.22,-4.054629


In [515]:
# crawford mask
crawford_mask = (result['County'] == 'Crawford')
result[crawford_mask].sort_values('growth_%', ascending=False)

,broad_type,County,2015 Q1 Total,2016 Q1 Total,growth_%
685,WHISKEY,Crawford,7415.02,24391.34,228.945033
586,VODKA,Crawford,2137.07,3557.53,66.467640
497,TEQUILA,Crawford,1388.66,1943.42,39.949304
22,BRANDY,Crawford,512.27,575.27,12.298202
306,RUM,Crawford,2713.76,2923.59,7.732077
208,OTHER,Crawford,3389.33,3332.14,-1.687354
403,SCHNAPP,Crawford,876.92,819.48,-6.550198
113,GIN,Crawford,390.87,287.36,-26.481951


# RECOMMENDATION

If by alcohol type, I would still recommend Fayette because the total sales is still greater than the growth rate % than the top 10 overall. Although over counties had growth rates in the 1000s, most of them are double digit sales jumping into triple digit sales. In a 3 month period, that is still not a good result. Whiskey in both Fayette and Crawford have 15000 at growth rate 99% and 24000 at growth rate 228%, respectively. If the investor would want to open a new store at either of these locations and only want to sell 1 type of alcohol, I would recommend Crawford and Whiskey.

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 7. Time-related effects

---

You could imagine that liquor sales might be affected by a variety of effects related to time. Do people buy more beer in the summer? Do liquor sales skyrocket in december? Do people buy less liquor on Tuesdays?

You have the date of sales in your dataset, which you can use to pull out time components.

1. Come up with 3 different hypotheses about how liquor sales may vary with time-related variables. 
2. Create a visualization exploring each hypothesis.
3. Write brief concluding remarks on what you observed.


### Hypothesis 1:
There is a 20% increase in sales during Q4 2015 compared to Q2 2015.

### Hypothesis 2:
Whiskey is equally consumed (Bottles Sold) throughout the entire 2015 year with a 3% standard deviation from the mean.

### Hypothesis 3:
Tequila has a growth rate of 25% from Q1 2015 to Q1 2016.


In [527]:
# Hypothesis 1 - Sale (Dollars), 2015 Q4 vs 2015 Q2
q4_mask = (iowa['Date'] < '2016-01-01') & (iowa['Date'] >= '2015-10-01')
q2_mask = (iowa['Date'] < '2015-07-01') & (iowa['Date'] >= '2015-4-01')

In [537]:
iowa_q4_tot = iowa[q4_mask].groupby(['County'])[['Sale (Dollars)']].sum().reset_index()
iowa_q2_tot = iowa[q2_mask].groupby(['County'])[['Sale (Dollars)']].sum().reset_index()

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 8. Appropriate spending

---

The owner you worked with before realized that opening new stores cost money, something he had completely overlooked earlier. He brings you back in to help him figure out how much he should be willing to spend on opening his store.

### 8.1 Calculate costs and time to recoup.

The owner has decided he will buy 20,000 gallons of the liquor type you recommended (randomly sampled across the different brands/subtypes). He will also be opening his store in the area you recommended.

The cost of buying the property for his store will be $150,000.

1. How much should he expect to spend on the 20,000 gallons of liquor alone?
2. Based on the amount he's spent on the liquor and property, and on the profit per time period he is expected to have, how long will it take him to recoup the costs?

<img src="http://imgur.com/gGrau8s.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">


### 8.2 Calculate maximum loan interest rate.

The owner unfortunately has to take out a loan to to pay for the entirety of the property and liquor. 

**Conditions of the loan**:

Every quarter, (except for Q1 when he takes out the loan), the interest rate is applied to the remaining balance on the loan and added to his debt. 

If, for example, he made no payments whatsoever, not only would the amount of debt he owed keep increasing by quarter but the debt incurred from interest would keep increasing as well. (Since his debt keeps growing, the interest rate is applied each month to the bigger balance, incurring more debt each time.)

**Determining maximum acceptable interest:**

Given this, _the owner wants to be 99% confident that he will be able to pay off his loans._ If the interest rate is too high he will have to eventually declare bankruptcy.

Based on his expected profits per quarter (assume the expected profit stays the same for all future quarters), what is the maximum interest rate he should accept on the loan to be 99% confident he will be able to pay it off?